In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# API URL
url_KRX = "http://data-dbg.krx.co.kr/svc/apis/idx/krx_dd_trd"
# 인증키 설정
auth_key = "956087495A1D4769A88B0F3411D0890EF02445EB"
# 헤더작성
headers = {"AUTH_KEY": auth_key,}

In [3]:
def get_monthly_data(year, month, whatKRX):
    # 시작일과 종료일 계산
    start_date = datetime(year, month, 1)
    if month == 12:
        end_date = datetime(year + 1, 1, 1) - timedelta(days=1)
    else:
        end_date = datetime(year, month + 1, 1) - timedelta(days=1)

    all_data = []
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")
        params_KRX = {"basDd": date_str}
        response_KRX = requests.get(url_KRX, headers=headers, params=params_KRX)

        if response_KRX.status_code == 200:
            data = response_KRX.json()
            if 'OutBlock_1' in data and data['OutBlock_1']:  # 데이터가 존재하는지 확인
                df = pd.DataFrame(data['OutBlock_1'])
                all_data.append(df)
        else:
            print(f"Error {response_KRX.status_code} on {date_str}: {response_KRX.text}")

        current_date += timedelta(days=1)

    if all_data:
        monthly_df = pd.concat(all_data, ignore_index=True)
        monthly_df = monthly_df.rename(columns={
        'BAS_DD': '기준일자',
        'IDX_CLSS': '계열구분',
        'IDX_NM': '지수명',
        'CLSPRC_IDX': '종가',
        'CMPPREVDD_IDX': '대비',
        'FLUC_RT': '등락률',
        'OPNPRC_IDX': '시가',
        'HGPRC_IDX': '고가',
        'LWPRC_IDX': '저가',
        'ACC_TRDVOL': '거래량',
        'ACC_TRDVAL': '거래대금',
        'MKTCAP': '상장시가총액'
        })
        monthly_df = monthly_df[monthly_df['지수명'] == whatKRX]
        monthly_df = monthly_df.reset_index(drop=True)
        return monthly_df
    else:
        return None

In [4]:
def get_yearly_data(year,whatKRX):
    all_data = []
    for month in range(1, 13):
        start_date = datetime(year, month, 1)
        if month == 12:
            end_date = datetime(year + 1, 1, 1) - timedelta(days=1)
        else:
            end_date = datetime(year, month + 1, 1) - timedelta(days=1)

        current_date = start_date
        while current_date <= end_date:
            date_str = current_date.strftime("%Y%m%d")
            params_KRX = {"basDd": date_str}
            response_KRX = requests.get(url_KRX, headers=headers, params=params_KRX)

            if response_KRX.status_code == 200:
                data = response_KRX.json()
                if 'OutBlock_1' in data and data['OutBlock_1']:
                    df = pd.DataFrame(data['OutBlock_1'])
                    all_data.append(df)
            else:
                print(f"Error {response_KRX.status_code} on {date_str}: {response_KRX.text}")

            current_date += timedelta(days=1)

    if all_data:
        yearly_df = pd.concat(all_data, ignore_index=True)
        yearly_df = yearly_df.rename(columns={
        'BAS_DD': '기준일자',
        'IDX_CLSS': '계열구분',
        'IDX_NM': '지수명',
        'CLSPRC_IDX': '종가',
        'CMPPREVDD_IDX': '대비',
        'FLUC_RT': '등락률',
        'OPNPRC_IDX': '시가',
        'HGPRC_IDX': '고가',
        'LWPRC_IDX': '저가',
        'ACC_TRDVOL': '거래량',
        'ACC_TRDVAL': '거래대금',
        'MKTCAP': '상장시가총액'
        })
        yearly_df = yearly_df[yearly_df['지수명'] == whatKRX]
        yearly_df = yearly_df.reset_index(drop=True)
        return yearly_df
    else:
        return None

In [6]:
monthly_df = get_monthly_data(2018, 3, 'KRX 헬스케어')
monthly_df.head()

,기준일자,계열구분,지수명,종가,대비,등락률,시가,고가,저가,거래량,거래대금,상장시가총액
0,20180302,KRX,KRX 헬스케어,4763.60,112.69,2.42,4610.06,4763.60,4608.76,24401497,1981993589320,176326497969560
1,20180305,KRX,KRX 헬스케어,4736.08,-27.52,-0.58,4841.03,4856.16,4672.91,23273580,2362559243155,175238532015130
2,20180306,KRX,KRX 헬스케어,4764.91,28.83,0.61,4788.78,4810.18,4677.98,18679891,1753840298360,175706920750160
3,20180307,KRX,KRX 헬스케어,4499.19,-265.72,-5.58,4640.48,4694.51,4498.17,36543570,4445061041363,166469012325715
4,20180308,KRX,KRX 헬스케어,4655.41,156.22,3.47,4575.43,4655.68,4549.50,35143365,3916458135810,171868133758380
